# Gene tree inference

Following the terminology used in *ipcoal*, gene trees refer to inferred hypotheses for the true genealogical relationships shared by a set of sampled genomes. Gene trees can be inferred from the sequence that evolve along the edges of genealogies. Our purpose here is to infer gene trees and compare them to the true genealogies.

The `ipcoal.phylo` module can be used to implement common gene tree inference methods to simulated sequences under a variety of approaches to investigate sources of phylogenetic error. Some examples might include investigating how different demographic and simulation parameters affect sequence variation; how substitution models affect gene tree inference; how phased haploid versus diploid (ambiguity character) sequences affect accuracy; how separating sequences between recombination events versus concatenating them affects gene tree error; and many more. 

In [114]:
import ipcoal
import toytree
import msprime
import numpy as np

In [3]:
# ipcoal.phylo.infer_raxml_ng_tree
# ipcoal.phylo.infer_raxml_ng_trees

### tldr;
Use `ipcoal.phylo.infer_raxml_ng_tree` to infer a gene tree from sequence data, or `ipcoal.phylo.infer_raxml_ng_trees` to return a dataframe with a collection of inferred trees from a set of sequences. These each take an `ipcoal.Model` class as input, and can select one or more loci to analyze individually, or as a concatenated sequence. In this example we infer gene trees for diploid sequences from 5 loci first as one large concatenated locus, and then also individually as separate loci.

In [4]:
# simulate sequences for two populations
model = ipcoal.Model("(a:50000,b:50000);", Ne=20000, nsamples=8, recomb=0, seed_mutations=123, seed_trees=123)
model.sim_loci(nloci=10, nsites=10000)

In [5]:
# infer a gene tree for loci 0-5 concatenated and draw the gtree
gtree = ipcoal.phylo.infer_raxml_ng_tree(model, idxs=[0, 1, 2, 3, 4], diploid=True, nboots=100)
gtree.mod.root_on_minimal_ancestor_deviation(inplace=True)
gtree.draw(ts='b', scale_bar=True, label="concat loci 0-5");

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t82149144573b4594b7428835daf6bfad"> b_1 b_3 b_0 b_2 a_3 a_1 a_0 a_2 0.00075 0.0005 0.00025 0 concat loci 0-5

In [6]:
# infer gene trees separately for loci 0-5 as a dataframe
gtdf = ipcoal.phylo.infer_raxml_ng_trees(model, idxs=range(5), diploid=True, nboots=100)
gtdf

,locus,start,end,nbps,nsnps,gene_tree
0,0,0,10000,10000,74,"((b_3:1e-06,(b_0:1e-06,b..."
1,1,0,10000,10000,34,"(a_3:1e-06,a_0:1e-06,((a..."
2,2,0,10000,10000,24,"(b_1:1e-06,b_0:1e-06,(b_..."
3,3,0,10000,10000,38,"((b_0:1e-06,(b_3:0.0003,..."
4,4,0,10000,10000,40,"(b_2:1e-06,b_1:0.000899,..."


## Methods

### infer_raxml_ng_tree_from_alignment
The function `infer_raxml_ng_tree_from_alignment` takes an alignment `str` as input and returns a `ToyTree` inferred by `raxml-ng`. You can toggle many arguments to `raxml-ng` using this function, but not all. It's primary utility is to provide a convenient shortcut to calling `raxml-ng` from Python which can also clean up the temporary files for you. This is the simplest of the three methods shown here. The examle below provides a phylip-formatted alignment as input and draws the inferred gene tree.

In [7]:
# a phylip string
phy = """\
4 10
A ACAACCGGTT
B ACATCCGGAA
C CCATCCGGAT
D AACACCGGTT
"""

# call raxml-ng on phylip str and draw result ToyTree
tree = ipcoal.phylo.infer_raxml_ng_tree_from_alignment(phy)
tree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0637aaff952b42e68741797320f44fd4"> A D B C

### infer_raxml_ng_tree
The function `infer_raxml_ng_tree` provides quite a bit more functionality than the one above. This takes an `ipcoal.Model` object as input and provides a number of arguments for extracting one or more simulated loci from the object and optionally concatenating them (using the `idxs` arg), and an option to join haploid sequences into pairs as diploid sequences (using `diploid=True`). You can also toggle options to `raxml-ng`, including fine-tuning the parallelization. Under the hood, this function writes a phylip formatted sequence file to a temporary directory and calls raxml on the tmp file, then removes it. You can optionally set the `tmpdir` as an argument.

In [8]:
# concatenate loci 2-4 and infer gene tree
tree = ipcoal.phylo.infer_raxml_ng_tree(model, idxs=[2, 3, 4], diploid=True)
tree.draw(scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf09c2b2821e34d9a9cca0887e50109c8"> b_2 b_3 b_1 a_1 a_3 a_0 a_2 b_0 0.0012 0.0008 0.0004 0

### infer_raxml_ng_trees
The function `infer_raxml_ng_trees` is similar to the one above but in addition to the `raxml-ng` parallelization arguments `nthreads` (used per tree) and `nworkers` (num parallel boots per tree); it also includes an ipcoal-level parallelization option `njobs` for distributing the individual `infer_raxml_ng_tree` jobs in parallel. If you are working on a large cluster and running many thousands of small jobs you can use this to achieve much greater parallelization. Note, `raxml-ng` will raise an error if you request more resources among concurrently running jobs than are available. You should very much try to avoid this, as it slows things down greatly. But, if it occurs rarely during a job and you want to avoid the error you can use the options `do_not_autoscale_threads` and `perf_threads`. 

In [9]:
# return a pandas Dataframe w/ trees and stats
ipcoal.phylo.infer_raxml_ng_trees(model, diploid=True, nthreads=2, nworkers=1, nproc=5)

,locus,start,end,nbps,nsnps,gene_tree
0,0,0,10000,10000,74,"(a_3:1e-06,a_0:0.000398,..."
1,1,0,10000,10000,34,"((b_3:1e-06,b_1:1e-06):0..."
2,2,0,10000,10000,24,"(b_2:1e-06,(b_1:1e-06,b_..."
3,3,0,10000,10000,38,"(a_3:1e-06,a_2:1e-06,a_0..."
4,4,0,10000,10000,40,"(b_3:1e-06,(a_2:1e-06,a_..."
5,5,0,10000,10000,53,"(b_0:1e-06,(a_0:1e-06,a_..."
6,6,0,10000,10000,52,"(a_2:1e-06,a_0:1e-06,((a..."
7,7,0,10000,10000,54,"(b_1:1e-06,b_0:1e-06,b_2..."
8,8,0,10000,10000,29,"(((b_2:0.0004,b_3:1e-06,..."
9,9,0,10000,10000,53,"(a_3:1e-06,a_1:1e-06,a_2..."


## Diploid sequences
The option `diploid=True` joins haploid sequences from within the same population together to create diploid sequences. Because samples within the same population are treated as randomly mating (panmictic) the order in which they are joined should not matter. Therefore, we simply join samples sequentially ordered by their alphanumeric tip labels. For example, "a_0" and "a_1" will be joined to create a new diploid "a_0"; and "a_2" and "a_3" would be joined to create the new "a_1", and so on with sequential numbering. The effect of analyzing diploid sequences will be generally greater when heterozygosity of samples is greater. 

In [10]:
# infer gene tree from diploid sequences
tree = ipcoal.phylo.infer_raxml_ng_tree(model, idxs=[1], diploid=True)
tree.root("~a").ladderize().draw(scale_bar=True);

# infer gene tree from diploid sequences
tree = ipcoal.phylo.infer_raxml_ng_tree(model, idxs=[1], diploid=False)
tree.root("~a").ladderize().draw(scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t23b09a61b80c454ba1c4b8793f37efac"> b_2 b_0 b_3 b_1 a_0 a_1 a_2 a_3 0.0008 0.0004 0

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="364.96px" viewBox="0 0 300.0 364.96" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t27dcede5406142fc892e23659e574c05"> b_0 b_5 b_1 b_4 b_7 b_2 b_3 b_6 a_3 a_1 a_5 a_6 a_0 a_7 a_4 a_2 0.0012 0.0008 0.0004 0

## Comparing trees
One of the core purposes of simulation is to compare inferred results to the ground truth. (See the Cookbooks/Examples section.) When simulating data we know the true genealogical history for our set of samples exactly at every position of their genomes. Therefore we can measure error as the differences between inferred gene trees and the true genealogies. One way to do this is to use tree distance metrics, many of which are implemented in *toytree*. 

In the example below we simulate a tree sequence across a 100Kb locus for 5-tip species tree and sample 2 genomes per lineage. This creates 449 genealogies.

In [101]:
# simulate sequences for two populations
sptree = toytree.rtree.unittree(ntips=5, treeheight=1e6, seed=123)
model = ipcoal.Model(sptree, Ne=400_000, nsamples=2, seed_mutations=123, seed_trees=123)
model.sim_loci(nloci=1, nsites=100_000)
model.draw_genealogies(shared_axes=True, scale_bar=100_000);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="250.0px" viewBox="0 0 750.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tef4f7617f24846269ff9ca2272212536"> r1_1 r4_0 r4_1 r3_0 r3_1 r1_0 r2_0 r2_1 r0_0 r0_1 0 5 10 15 r4_0 r4_1 r3_0 r3_1 r1_0 r2_0 r2_1 r1_1 r0_0 r0_1 0 5 10 15 r3_0 r3_1 r2_0 r2_1 r1_1 r0_0 r0_1 r1_0 r4_0 r4_1 0 5 10 15 r2_0 r2_1 r1_1 r0_0 r0_1 r4_1 r3_0 r3_1 r1_0 r4_0 0 5 10 15

In [103]:
# infer a concatenated gene tree for the locus
ctree = ipcoal.phylo.infer_raxml_ng_tree(model)

# draw the concatenation tree
ctree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc831590637574ecd8b3afd772646d157"> r0_0 r0_1 r1_1 r1_0 r2_0 r2_1 r4_1 r4_0 r3_0 r3_1

Here I use the `toytree.distance.get_treedist_rfg_mci` to return the Generalized Robinson-Foulds distance using Mutual Clustering Information (MCI) to measure the distance between the concatenation tree and each observed genealogy across the length of the simulated chromosome. 

In [104]:
# measure MCI RF dist for each gene tree
tdists = [ctree.distance.get_treedist_rfg_mci(i, normalize=True) for i in toytree.mtree(model.df.genealogy)]

We can then plot the tree distances along the length of the chromosome. You can see that for most parts of the genome the local genealogy does not match the concatenation tree topology. There is also a pattern of auto-correlation -- nearby regions of the genome tend to exhibit similar trees, and thus similar tree distances. 

In [120]:
import toyplot
canvas = toyplot.Canvas(height=300)
axes = canvas.cartesian(ylabel="Generalized RF distance", xlabel="Genome position")
axes.fill(model.df.start, tdists)
axes.x.ticks.locator = toyplot.locator.Extended(count=6)
axes.x.ticks.show = axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="300.0px" viewBox="0 0 600 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7f936bdb39ab4677b151fde5285de841"> <polygon points="50.0,249.99999999999994 50.035,249.99999999999994 52.370000000000005,249.99999999999994 52.54,249.99999999999994 52.849999999999994,249.99999999999994 53.214999999999996,249.99999999999994 53.265,249.99999999999994 53.279999999999994,249.99999999999994 53.574999999999996,249.99999999999994 53.945,249.99999999999994 54.120000000000005,249.99999999999994 54.54,249.99999999999994 56.81,249.99999999999994 57.239999999999995,249.99999999999994 60.839999999999996,249.99999999999994 60.95,249.99999999999994 61.42,249.99999999999994 64.465,249.99999999999994 64.80000000000001,249.99999999999994 65.225,249.99999999999994 65.485,249.99999999999994 67.42,249.99999999999994 67.82,249.99999999999994 68.065,249.99999999999994 68.245,249.99999999999994 68.345,249.99999999999994 68.71,249.99999999999994 68.825,249.99999999999994 69.315,249.99999999999994 69.52,249.99999999999994 71.78,249.99999999999994 72.235,249.99999999999994 72.315,249.99999999999994 72.775,249.99999999999994 73.095,249.99999999999994 73.19,249.99999999999994 73.55,249.99999999999994 74.395,249.99999999999994 74.42500000000001,249.99999999999994 75.09,249.99999999999994 76.25,249.99999999999994 76.34,249.99999999999994 76.81,249.99999999999994 79.175,249.99999999999994 80.035,249.99999999999994 80.335,249.99999999999994 80.695,249.99999999999994 82.505,249.99999999999994 82.78,249.99999999999994 83.925,249.99999999999994 86.56,249.99999999999994 87.185,249.99999999999994 87.21,249.99999999999994 87.47,249.99999999999994 88.375,249.99999999999994 89.66,249.99999999999994 91.305,249.99999999999994 94.9,249.99999999999994 94.945,249.99999999999994 97.315,249.99999999999994 98.32499999999999,249.99999999999994 98.69,249.99999999999994 100.17,249.99999999999994 101.675,249.99999999999994 102.13,249.99999999999994 102.94,249.99999999999994 104.065,249.99999999999994 105.405,249.99999999999994 105.525,249.99999999999994 105.85,249.99999999999994 106.065,249.99999999999994 107.235,249.99999999999994 110.13499999999999,249.99999999999994 110.88499999999999,249.99999999999994 111.03999999999999,249.99999999999994 112.51500000000001,249.99999999999994 112.94,249.99999999999994 113.36500000000001,249.99999999999994 113.49000000000001,249.99999999999994 113.775,249.99999999999994 113.88,249.99999999999994 114.145,249.99999999999994 114.41499999999999,249.99999999999994 114.69999999999999,249.99999999999994 115.1,249.99999999999994 115.385,249.99999999999994 115.50999999999999,249.99999999999994 115.57499999999999,249.99999999999994 117.415,249.99999999999994 119.185,249.99999999999994 119.34,249.99999999999994 120.785,249.99999999999994 120.845,249.99999999999994 121.95999999999998,249.99999999999994 122.4,249.99999999999994 122.49000000000001,249.99999999999994 123.91,249.99999999999994 123.94,249.99999999999994 124.35500000000002,249.99999999999994 125.81,249.99999999999994 125.965,249.99999999999994 126.14000000000001,249.99999999999994 126.185,249.99999999999994 126.38999999999999,249.99999999999994 126.39500000000001,249.99999999999994 126.97,249.99999999999994 127.245,249.99999999999994 127.32500000000002,249.99999999999994 127.64,249.99999999999994 128.085,249.99999999999994 129.39499999999998,249.99999999999994 129.42000000000002,249.99999999999994 129.54500000000002,249.99999999999994 129.745,249.99999999999994 131.39000000000001,249.99999999999994 132.385,249.99999999999994 132.79,249.99999999999994 133.22,249.99999999999994 